<a href="https://colab.research.google.com/github/siting1206/Plant_seedlings_classification/blob/main/Plant_seedlings_classification(Swin)_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.utils.data.distributed
import torchvision.transforms as transforms
from PIL import Image
from torch.autograd import Variable
import os
import pandas as pd

classes = ('Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed',
           'Common wheat', 'Fat Hen', 'Loose Silky-bent',
           'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet')
transform_test = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.51819474, 0.5250407, 0.4945761], std=[0.24228974, 0.24347611, 0.2530049])
])

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=torch.load('checkpoints/SwinV2/best.pth')
model.eval()
model.to(DEVICE)

test_name, pred_name = [], []
path = 'plant-seedlings-classification/test/'
testList = os.listdir(path)
for file in testList:
    img = Image.open(path + file)
    img = transform_test(img)
    img.unsqueeze_(0)
    img = Variable(img).to(DEVICE)
    out = model(img)
    # Predict
    _, pred = torch.max(out.data, 1)
    test_name.append(file)
    pred_name.append(classes[pred.data.item()])
    # print('Image Name:{},predict:{}'.format(file, classes[pred.data.item()]))
print("Number of test images : ", len(test_name))

results = pd.DataFrame({"file":test_name, "species":pred_name})

results.head()

results.to_csv("submission_swin.csv",index=False)